# Alignment hypergraph

## Initialize

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext memory_profiler

import pprint
import operator # comparison operators lt and gt
from collections import defaultdict
from typing import Set, List
from dataclasses import dataclass
pp = pprint.PrettyPrinter(indent=2)

## Load data

In [2]:
%run create-blocks.ipynb # data will be in prioritized_blocks
# pp.pprint(prioritized_blocks) # take a look
# pp.pprint(token_to_block_dict) # and another look
# print(token_array, len(token_array)) # and one more look

{ 'w0': 'It has been disputed at what period of life the causes of '
        'variability, whatever they may be, generally act; whether during the '
        'early or late period of development of the embryo, or at the instant '
        "of conception. Geoffroy St. Hilaire's experiments show that unnatural "
        'treatment of the embryo causes monstrosities; and monstrosities '
        'cannot be separated by any clear line of distinction from mere '
        'variations. But I am strongly inclined to suspect that the most '
        'frequent cause of variability may be attributed to the male and '
        'female reproductive elements having been affected prior to the act of '
        'conception. Several reasons make me believe in this; but the chief '
        'one is the remarkable effect which confinement or cultivation has on '
        'the functions of the reproductive system; this system appearing to be '
        'far more susceptible than any other part of the organisation, 

## Create data structures

1. Create witness_node (dataclass) for witness nodes. Properties (not all may be needed):
    1. token string: str
    1. witness: str
    1. offset in witness: int
    1. hyperedge: hyperedge instance (initially null)
1. Create alignment_hyperedge (dataclass) for … wait for it! … alignment hyperedges! Properties:
    1. witness_nodes: set of witness_node objects
1. Create lists for witness_node and alignment_hyperedge instances
1. Create START and END nodes and associated hyperedges

In [3]:
@dataclass(unsafe_hash=True)
class WitnessNode:
    """Write a docstring some day"""
    token_string: str
    witness: str
    witness_offset: int
    hyperedge: hyperedge = None

    def __repr__(self):
        return self.token_string

In [4]:
class AlignmentHyperedge():
    """Write a real docstring someday"""
    def __init__(self, kwargs):
        self.__dict__.update(kwargs) # overwrite the real value
        self._sigla = [key for key in kwargs.keys()]
    def __repr__(self):
        return "|".join(
            [":".join(
                    [str(key), str(getattr(self, key))]
                ) for key in self._sigla]
        )
    def __setitem__(self, key, value):
        self._sigla.append(key)
        self.__dict__[key] = value
    def __getitem__(self, key):
        return self.__dict__[key]
    def __contains__(self, key):
        return key in self.__dict__
    def values(self):
        return [self[key] for key in self._sigla]
    def items(self):
        return [(key, self.__dict__[key]) for key in self._sigla]

In [5]:
# Store all witness nodes in dictionary of lists
witness_node_lists = defaultdict(list) # one per witness

In [6]:
# create START node, nodes for real tokens, and END node
for index, witness_siglum in enumerate(witness_sigla):
    witness_node_lists[witness_siglum].append(WitnessNode('START', witness_siglum, -1, None))
    for witness_token_offset, witness_token in enumerate(witnesses[index]): # list of tokens in a single witness
        witness_node_lists[witness_siglum].append(WitnessNode(witness_token, witness_siglum, witness_token_offset, None))
    witness_node_lists[witness_siglum].append(WitnessNode('END', witness_siglum, len(token_array), None))

In [7]:
# create hyperedges for START and END
hyperedges = []
offsets = [0, -1]
for offset in offsets:
    dictionary = {siglum: witness_node_lists[siglum][offset] for siglum in witness_sigla}
    hyperedge = AlignmentHyperedge(dictionary)
    for witness_node in dictionary.values():
        witness_node.hyperedge = hyperedge
    hyperedges.append(hyperedge)

In [8]:
def create_hyperedges(block):
    """Returns hyperedge data for block, together with offset_start_positions"""
    new_hyperedges = []
    for token_offset_in_block in range(selected_block.token_count):
        # first add the token_offset_in_block to the block start positions
        offset_start_positions = [value + token_offset_in_block for value in selected_block.all_start_positions]
        tokens_to_place = [
            (
                witness_sigla[token_membership_array[token_array_index]],
                token_witness_offset_array[token_array_index]
            )
         for token_array_index in offset_start_positions
#         if not placed_token_bitarray[token_array_index] # already placed, so don’t place it again
        ]
        dictionary = { # data for new hyperedge
            siglum: witness_node_lists[siglum][offset + 1]
            for siglum, offset
            in tokens_to_place
        }
        hyperedge = AlignmentHyperedge(dictionary)
        new_hyperedges.append(hyperedge)

    return (offset_start_positions, new_hyperedges)

In [9]:
def find_floor(siglum, new_offset):
    """Closest hyperedge to the left for a single witness"""
#     print("new_offset: "+ str(new_offset))
    for offset in range(new_offset - 1, -1, -1):
#         print("traversing: "+str(offset))
        # walk backwards looking for node with hyperedge
        node = witness_node_lists[siglum][offset]
#         print("Witness Node: "+str(node))
        if node.hyperedge:
#             print("hyperedge: "+str(node.hyperedge))
            return witness_node_lists[siglum][offset].hyperedge
    raise Exception("We should not get here!")

In [10]:
def find_ceiling(siglum, new_offset):
    """Closest hyperedge to the right for a single witness"""
    for offset in range(new_offset + 1, len(token_array)):
        # walk forwards looking for node with hyperedge
        if witness_node_lists[siglum][offset].hyperedge:
            return witness_node_lists[siglum][offset].hyperedge
    raise Exception("We should not get here!")

## Prose statement of transposition detection

1. Find floor and ceiling for block, where both floor and ceiling are sets of hyperedges, with each member of the set selected on the basis of just one witness.
1. For each hyperedge in the floor, verify that all nodes that belong to that hyperedge are to the left of the leftmost node for that witness in the block we are trying to place. If so, there is no transposition. If that is not the case, at least one witness would be transposed, so do nothing.
1. For each hyperedge in the ceiling, verify that all nodes that belong to that hyperedge are to the right of th rightmost node for that witness in the block we are trying to place. If so, there is no transposition. If that is not the case, at least one witness would be transposed, so do nothing.

In [11]:
def check_for_crossing_hyperedges(
    hyperedge_to_test: AlignmentHyperedge,
    boundary_hyperedges: List[AlignmentHyperedge], # upper or lower
    lower_or_higher: str) -> bool: # last parameter must be 'floor' or 'ceiling'
    """ Check inside floor and ceiling for consistency

    * Is every floor hyperedge to the left of every leftmost block node?
    * And similarly for the ceiling and the right edge?

    Same function checks 'floor' and 'ceiling' (raise on wrong argument)
    Return True iff no crossing hyperedges, i.e., no transposition
    Relies on operator module for gt and lt; syntax is operator.lt(a, b)

    """
    # Set comparison operator to lt or gt
    if lower_or_higher == 'floor':
        comp = operator.lt
    elif lower_or_higher == "ceiling":
        comp = operator.gt
    else:
        raise Exception("Third argument to check_for_crossing_hyperedges() must be 'floor' or 'ceiling'")

    for siglum, witness_node in hyperedge_to_test.items():
        for boundary_hyperedge in boundary_hyperedges:
            if siglum not in boundary_hyperedge._sigla:
                continue
            elif not comp(boundary_hyperedge[siglum].witness_offset, witness_node.witness_offset):
                return False;
    return True # if we're still alive at the end of the function

In [12]:
def check_for_transposition(hyperedges_to_check: List[AlignmentHyperedge]) -> bool:
    """Check for transposition

    List of hyperedges is consecutive because tokens are consecutive in blocks
    Floor of first and ceiling of last are boundaries for the entire list
    """
    first_hyperedge = hyperedges_to_check[0]
    last_hyperedge = hyperedges_to_check[-1]
    # print("\n")
    # print(f"{first_hyperedge=}")
    # print(f"{last_hyperedge=}")
    # Find first hyperedge to left for each witness
    #   and determine whether there is a transposition.
    #   They may not be the same hyperedge (!)
    lower_hyperedges = set()
    for siglum, witness_node in first_hyperedge.items():
        # position of node for each witness in first_hyperedge
        # convert from token offset to node offset
        lower_hyperedges.add(find_floor(siglum, witness_node.witness_offset +1))
    # Find ceiling for all witnesses in second hyperedge
    upper_hyperedges = set()
    for siglum, witness_node in last_hyperedge.items():
        # position of node for each witness in last_hyperedge
        # convert from token offset to node offset
        upper_hyperedges.add(find_ceiling(siglum, witness_node.witness_offset +1))
    # we now have: lower_hyperedges, upper_hyperedges
    # Check inside floor and ceiling for consistency; is every floor hyperedge to the left
    #   of every leftmost block node, and similarly for the ceiling and the right edge?
    floor_ok = check_for_crossing_hyperedges(first_hyperedge, lower_hyperedges, 'floor')
    ceiling_ok = check_for_crossing_hyperedges(last_hyperedge, upper_hyperedges, 'ceiling')
    return floor_ok and ceiling_ok # returns true only if both edges show no transposition

## Augmenting an existing hyperedge

* Every witness node always contains zero or one hyperedge. If zero, create new. If one, extend.
* Provisionally, check for two or more and raise an exception. Our assumption (okay, hope) is that it won't happen.

In [13]:
def merge_hyperedge(potential_hyperedge):
            # check whether a witness_node has got the hyperedge property set
            # a hyperedge may contain zero, some, or all witnesses; we need to handle all these cases
            found = set([witnessnode.hyperedge for siglum, witnessnode in potential_hyperedge.items() if witnessnode.hyperedge])
            if len(found) > 1:
                # found two or more existing hyperedges...
                # remove them from the hyperedges set
                for existing_hyperedge in found:
                    hyperedges.remove(existing_hyperedge)
                # replace them by a new hyperedge
                hyperedges.append(potential_hyperedge)
                # update hyperedge property on the witness nodes involved in the hyperedge
                new_hyperedge = potential_hyperedge
                for witness_node in new_hyperedge.values():
                    witness_node.hyperedge = new_hyperedge

#                 print(found) # debug only
#                 raise Exception('Eek! Nodes belong to different existing hyperedges!')
            elif len(found) == 0:
                # we checked for transpositions and we filtered out hyperedges for witness nodes that are already placed
                hyperedges.append(potential_hyperedge)
                # update hyperedge property on the witness nodes involved in the hyperedge
                new_hyperedge = potential_hyperedge
                for witness_node in new_hyperedge.values():
                    witness_node.hyperedge = new_hyperedge
            else: # update single existing hyperedge
                for siglum in potential_hyperedge._sigla:
                    list(found)[0][siglum] = potential_hyperedge[siglum] # add node to hyperedge (possibly redundantly)
                    potential_hyperedge[siglum].hyperedge = list(found)[0] # and update hyperedge property of node

In [14]:
# create all hyperedges (gingerly)
selected_blocks = filter(lambda x: len(x.all_start_positions) == x.witness_count, prioritized_blocks)
for selected_block in selected_blocks:
    # %memit pass
    offset_start_positions, new_hyperedges = create_hyperedges(selected_block)
    no_transposition = check_for_transposition(new_hyperedges)
    if no_transposition: # all-or-nothing
        for potential_hyperedge in new_hyperedges:
            merge_hyperedge(potential_hyperedge)
#      else:
#         print("Block "+str(selected_block)+" is filtered out because of perceived transposition")
#         raise Exception("Stop!")

# take a look
# pp.pprint(hyperedges)

In [15]:
# pp.pprint(witness_node_lists)
# pp.pprint(hyperedges)

## Visualize alignment hypergraph

In [16]:
# recover memory
selected_blocks = None
prioritized_blocks = None
raw_data_dict = None
lcp_array = None
suffix_array = None
token_array = None
token_membership_array = None
token_to_block_dict = None
token_witness_offset_array = None
witnesses = None

In [17]:
import graphviz
from IPython.display import SVG
from collections import defaultdict

# node id values must be strings for graphviz
a = graphviz.Digraph(format="svg")
a.attr(rankdir = "TB")
a.attr(rank = 'same')
a.attr(compound='true')

# create nodes and regular edges
for siglum, witness_node_list in witness_node_lists.items():
    c = graphviz.Digraph(siglum+'child') # can't reuse subgraph label, so prepend siglum
    c.attr(rankdir = "LR")
    c.attr(rank='same')
    for index,witness_node in enumerate(witness_node_list):
        node_id = "#".join((witness_node.witness, str(witness_node.witness_offset)))
        preceding_node_id = "#".join((witness_node_list[index - 1].witness, str(witness_node_list[index - 1].witness_offset)))
        c.node(node_id, "".join((node_id, '\n', witness_node.token_string)))
        if witness_node.token_string != 'START':
            c.edge(preceding_node_id, node_id)
    a.subgraph(c)

# add hyperedges (fake: connect only closest witnesses, pairwise)
for hyperedge in hyperedges: # find all sigla, in order, on hyperedge
    hyperedge_sigla = hyperedge._sigla
    filtered_sigla = list(filter(lambda x: x in hyperedge_sigla, witness_sigla)) # sigla in hyperedge, in order
    # print(hyperedge)
    for source_siglum, target_siglum in zip(filtered_sigla, filtered_sigla[1:]): # sigla, not nodes!
        source_node = hyperedge[source_siglum]
        target_node = hyperedge[target_siglum]
        if source_node.token_string in ("START", "END"): # force alignment by weighting start and end edges
            edge_weight="100"
        else:
            edge_weight="1"
        a.edge(
            "#".join((source_siglum, str(source_node.witness_offset))),
            "#".join((target_siglum, str(target_node.witness_offset))),
            weight=edge_weight
              )
SVG(a.view())

Unable to init server: Could not connect: Connection refused

(eog:17335): Gtk-WARNING **: 20:02:22.484: cannot open display: :0



(inkscape:17343): Gtk-WARNING **: 20:02:22.770: cannot open display: :0


Opening "Digraph.gv.svg" with GNU Image Manipulation Program  (image/svg+xml)


Cannot open display: 


In [18]:
SVG(a.view())

[17418:17418:0904/200224.961119:ERROR:browser_main_loop.cc(1412)] Unable to open X display.
Unable to init server: Could not connect: Connection refused
Error: cannot open display: :0
/usr/bin/xdg-open: 869: iceweasel: not found
/usr/bin/xdg-open: 869: seamonkey: not found
/usr/bin/xdg-open: 869: mozilla: not found
/usr/bin/xdg-open: 869: epiphany: not found
/usr/bin/xdg-open: 869: konqueror: not found
/usr/bin/xdg-open: 869: chromium: not found
/usr/bin/xdg-open: 869: chromium-browser: not found


[17474:17474:0904/200225.666275:ERROR:browser_main_loop.cc(1412)] Unable to open X display.


>
inkscape /tmp/lynxXXXX38q5f5/L17508-4019TMP.svgz

lynx: Start file could not be found or is not text/html or text/plain
      Exiting...



(inkscape:17516): Gtk-WARNING **: 20:02:25.962: cannot open display: :0
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found


>
inkscape /tmp/lynxXXXX4Swh5B/L17528-9921TMP.svgz

lynx: Start file could not be found or is not text/html or text/plain
      Exiting...



(inkscape:17536): Gtk-WARNING **: 20:02:26.182: cannot open display: :0


t-family="Times,serif" font-size="14.00">w2#202</text>
<text text-anchor="middle" x="25574.96" y="-284.89" font-family="Times,serif" font-size="14.00">their</text>
</g>
<!-- w1#202&#45;&gt;w2#202 -->
<g id="edge3381" class="edge">
<title>w1#202&#45;&gt;w2#202</title>
<path fill="none" stroke="black" d="M25574.96,-358.51C25574.96,-350.54 25574.96,-341.65 25574.96,-333.16"/>
<polygon fill="black" stroke="black" points="25578.46,-333.09 25574.96,-323.09 25571.46,-333.09 25578.46,-333.09"/>
</g>
<!-- w1#203&#45;&gt;w1#204 -->
<g id="edge656" class="edge">
<title>w1#203&#45;&gt;w1#204</title>
<path fill="none" stroke="black" d="M25756.53,-385.83C25759.13,-385.83 25761.73,-385.83 25764.33,-385.83"/>
<polygon fill="black" stroke="black" points="25764.36,-389.33 25774.36,-385.83 25764.36,-382.33 25764.36,-389.33"/>
</g>
<!-- w2#203 -->
<g id="node1106" class="node">
<title>w2#203</title>
<ellipse fill="none" stroke="black" cx="25701.96" cy="-296.09" rx="54.39" ry="26.74"/>
<text text-anchor="m